# Notebook for a space-time discretization

In this notebook we are treating following problem 
$$
u_t - \Delta u + \mathbf{w} \cdot \nabla u = f \quad \text{in } \Omega  = [0,1] \times [0,T].
$$
with the initial/boundary conditions
$$
u(x,0) = \sin(\pi x), \quad u(0,t) = u(1,t)=0.
$$
The weakformulation of the problem in the CG setting is

Find a function $u_0 \in H^1_0(\Omega)$ , such that  for all  $v \in H^1_0(\Omega) $:

$$
\int_{\Omega} (u_t)_0 v \,dx \,dt
+ \int_{\Omega} \nabla u_0 \cdot \nabla v \,dx \,dt
+ \int_{\Omega} (\mathbf{w} \cdot \nabla u_0) v \,dx \,dt
= \int_{\Omega} f v \,dx \,dt
- \int_{\Omega} u_D v \,dx \,dt.
$$

where we used the decomposition of $u = u_D + u_0$

For $w = 0$ (the heatequation), we have for the chosen initial/boundary conditions following analytical solution 

$u(x,t)= \sin(\pi x) e^{-t \pi^2}$

In [1]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
from netgen.occ import *

T = 0.5 # length of the time interval
shape = Rectangle(1,T).Face()
shape.edges.Min(X).name="left"
shape.edges.Max(X).name="right"
shape.edges.Min(Y).name="bottom"
shape.edges.Max(Y).name="top"
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.25))
Draw(mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [4]:
deformation = GridFunction(H1(mesh,order=1,dim=mesh.dim))
deformation.Set(CF((x,2*y))-CF((x,y)))
Draw(deformation,mesh)
mesh.SetDeformation(deformation)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [5]:
Draw(sin(x+y),mesh,deformation=deformation)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene